In [1]:
import warnings
warnings.filterwarnings('ignore') # Filter out warnings
import numpy as np
import pandas as pd
import random as rnd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from scipy import stats
import math
from math import isnan

In [2]:
## FOR STATISTICAL ANALYSIS
# Create a Combined Dataframe from message csv and orderbook csv
def Merge_MsgAndOrdb(msg_path, ordb_path):
    # get the level of the LOBSTER dataset
    level = int(msg_path.split(".")[0][-1])
    # get the date
    date = msg_path.split(".")[0].split("_")[1]

    # add header row for message csv
    msg_names = ["Time_stamp", "Type", "OrderID", "V", "P", "Dir"]  # V: size; P: price; Dir: direction
    df_msg = pd.read_csv(msg_path, names=msg_names)
    # add header row for orderbook csv
    default_ordb_names = ['P_ask', 'V_ask', 'P_bid', 'V_bid']
    ordb_names = []
    for i in range(level):
        for item in default_ordb_names:
            ordb_names.append(str(item) + str("_") + str(i + 1))
    df_ordb = pd.read_csv(ordb_path, names=ordb_names)

    # adding a meaningful time column
    df_msg['Time'] = pd.to_datetime(df_msg['Time_stamp'], unit="s", origin=pd.Timestamp(date))

    # combine two dataframes and return it
    df = pd.concat([df_msg, df_ordb], axis=1)
    return df

# Renaming columns as to strip out redundnat words
def RenameColumn(df,state_type):
    old_col = df.columns.tolist()
    new_col = []
    for col in old_col:
        if ( ("bid" in col) | ("ask" in col) ):
            new_col.append(col+str(" ")+state_type)
        elif ((col == 'V') | (col == 'P')):
            new_col.append(col+str(" ")+state_type)
        elif (col=='Time'):
            new_col.append(col+str(" ")+state_type)
        else:
            new_col.append(col)
    columns = dict(zip(old_col,new_col))
    df.rename(columns=columns,inplace=True)
    return

def GroupByOrderType(df,typeID):                        # for now, typeID should only be 2 or 3 or 4 or 5
    # lookup dictionary
    lookup = {1:'Sub', 2:'Can', 3:'Del', 4:'ExVis',5:'ExHid',7:'Halt'}
    # getting all the orders of type typeID
    orderID_set = set(df[(df['Type']==typeID)].OrderID.tolist())
    df_subset = df.loc[df['OrderID'].isin(orderID_set)]
    
    # get initial states when orders of this type is submitted
    df_init = df_subset[(df_subset['Type']==1)]
    RenameColumn(df_init,state_type=lookup[1])
    
    # get the end states of type typeID
    df_end = df_subset[(df_subset['Type']==typeID)]
    RenameColumn(df_end,state_type=lookup[typeID])
    
    # merge these two dataframes
    df = df_init.merge(df_end, left_on='OrderID', right_on='OrderID', how='outer')

    return df

In [3]:
## FOR PAPER 462
# Compute Time-insensitive variables
def ComputeTimeInsenstiveSet(df_original, levels=5):
    df = df_original.copy(deep=False)
    for i in range(1,levels+1):
        # compute bid-ask spreads and mid-prices (v2)
        df["bid-ask spread {}".format(i)] = df["P_ask_{}".format(i)] - df["P_bid_{}".format(i)]
        df["mid-price {}".format(i)] = (df["P_ask_{}".format(i)] + df["P_bid_{}".format(i)])/2
        # compute price differences (v3)
        df["d_P_ask_{}{}".format(levels,1)] = df["P_ask_{}".format(levels)] - df["P_ask_1"]
        df["d_P_bid_{}{}".format(levels,1)] = df["P_bid_{}".format(levels)] - df["P_bid_1"]
        for j in range(1,levels):
            df["d_P_ask_{}{}".format(j+1,j)] = np.abs(df["P_ask_{}".format(j+1)] - df["P_ask_{}".format(j)])
            df["d_P_bid_{}{}".format(j+1,j)] = np.abs(df["P_bid_{}".format(j+1)] - df["P_bid_{}".format(j)])
        # compute mean prices and volumes (v4)
        df["Mean_P_ask"] = df[["P_ask_{}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        df["Mean_P_bid"] = df[["P_bid_{}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        df["Mean_V_ask"] = df[["V_ask_{}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        df["Mean_V_bid"] = df[["V_bid_{}".format(i) for i in range(1,levels+1)]].mean(axis=1)
        # compute accumulated differences (v5)
        df["P_accu"] = df[["P_ask_{}".format(i) for i in range(1,levels+1)]].sum(axis=1) \
            - df[["P_bid_{}".format(i) for i in range(1,levels+1)]].sum(axis=1)
        df["V_accu"] = df[["V_ask_{}".format(i) for i in range(1,levels+1)]].sum(axis=1) \
            - df[["V_bid_{}".format(i) for i in range(1,levels+1)]].sum(axis=1)
    # returning...
    return df

# Compute Time-sensitive variables
def ComputeTimeSensitiveSet(df_original,levels=5,dt_secs=1):
    df = df_original.copy(deep=False)
    #
    for index, row in df.iterrows():
        t = row['Time_stamp']
        temp = df.loc[(df['Time_stamp']>t-dt_secs)&(df['Time_stamp']<=t)] # for v6
        # Note: direction = 1 <--> buy/bid order;   direction = -1 <--> sell/ask order
        temp_la = df.loc[(df['Time_stamp']>t-dt_secs)&(df['Time_stamp']<=t)&(df['Dir']==-1)] # limit ask order
        temp_lb = df.loc[(df['Time_stamp']>t-dt_secs)&(df['Time_stamp']<=t)&(df['Dir']==1)]  # limit bid order
        temp_ca = df.loc[(df['Time_stamp']>t-dt_secs)&(df['Time_stamp']<=t)
                         &(df['Dir']==-1)&(df['Type']==2)] # cancelled limit ask order
        temp_cb = df.loc[(df['Time_stamp']>t-dt_secs)&(df['Time_stamp']<=t)
                         &(df['Dir']==1)&(df['Type']==2)]  # cancelled limit buy order
        for i in range(1,levels+1):
            # compute price and volume derivatives (v6)
            df.set_value(index,'der_P_ask_{}'.format(i),
                         (temp['P_ask_{}'.format(i)].iloc[-1]-temp['P_ask_{}'.format(i)].iloc[0])/dt_secs)
            df.set_value(index,'der_P_bid_{}'.format(i),
                         (temp['P_bid_{}'.format(i)].iloc[-1]-temp['P_bid_{}'.format(i)].iloc[0])/dt_secs)
            df.set_value(index,'der_V_ask_{}'.format(i),
                         (temp['V_ask_{}'.format(i)].iloc[-1]-temp['V_ask_{}'.format(i)].iloc[0])/dt_secs)
            df.set_value(index,'der_V_bid_{}'.format(i),
                         (temp['V_bid_{}'.format(i)].iloc[-1]-temp['V_bid_{}'.format(i)].iloc[0])/dt_secs)
            # compute average intensity of each type (v7)
            df.set_value(index,'lam_la',int(len(temp_la.index)/dt_secs))
            df.set_value(index,'lam_lb',int(len(temp_lb.index)/dt_secs))
            df.set_value(index,'lam_ca',int(len(temp_ca.index)/dt_secs))
            df.set_value(index,'lam_cb',int(len(temp_cb.index)/dt_secs))
            # compute relative intensity indicators (v8) ???
            # compute accelerations (market/limit)  (v9) ???

    return df

In [4]:
df = Merge_MsgAndOrdb(msg_path='AMZN_2012-06-21_34200000_57600000_message_5.csv',
                     ordb_path='AMZN_2012-06-21_34200000_57600000_orderbook_5.csv')
df.head(5)

,Time_stamp,Type,OrderID,V,P,Dir,Time,P_ask_1,V_ask_1,P_bid_1,...,P_bid_3,V_bid_3,P_ask_4,V_ask_4,P_bid_4,V_bid_4,P_ask_5,V_ask_5,P_bid_5,V_bid_5
0,34200.017460,5,0,1,2238200,-1,2012-06-21 09:30:00.017460,2239500,100,2231800,...,2230400,100,2242500,100,2230000,10,2244000,547,2226200,100
1,34200.189608,1,11885113,21,2238100,1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2230700,200,2242500,100,2230400,100,2244000,547,2230000,10
2,34200.189608,1,3911376,20,2239600,-1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2230700,200,2240000,220,2230400,100,2242500,100,2230000,10
3,34200.189608,1,11534792,100,2237500,1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2231800,100,2240000,220,2230700,200,2242500,100,2230400,100
4,34200.189608,1,1365373,13,2240000,-1,2012-06-21 09:30:00.189608,2239500,100,2238100,...,2231800,100,2240000,233,2230700,200,2242500,100,2230400,100


### Adding the labels (lb, la, mb, ma) to classify all of the order

In [ ]:
total = len(df.index)
labels = []
for index, row in df.iterrows():
    if (((index)%65000 == 0)):
        print(index, "out of", total)
    if (row["Type"] == 1):
        if (row["Dir"] == -1):
            labels.append('la')
        elif (row["Dir"] == 1):
            labels.append('lb')
    elif (row["Type"] in [4, 5]):
        if (row["Dir"] == -1):
            labels.append('mb')
        elif (row["Dir"] == 1):
            labels.append('ma')
    elif (row["Type"] == 2):
        labels.append('pc')
    elif (row["Type"] == 3):
        labels.append('c')
    else:
        labels.append(None)

df["Labels"] = labels
#print(len(labels), total)

0 out of 155935
65000 out of 155935
130000 out of 155935


### Classify the level where action takes

In [ ]:
check = False
nlevels = 5
levels = [None]
for index, row in df.iterrows():
    if (index > 0):
        bid_levels = np.array(df.iloc[index-1][["P_bid_{}".format(i) for i in range(1,nlevels+1)]])
        ask_levels = np.array(df.iloc[index-1][["P_ask_{}".format(i) for i in range(1,nlevels+1)]])
        
        price = row["P"]
        if (row["Labels"] == 'c'):
            if (row["Dir"] == -1):
                index = np.where(price == ask_levels)[0]
                levels.append(int(index+1))
            elif (row["Dir"] == 1):
                index = np.where(price == bid_levels)[0]
                levels.append(int(index+1))
            else:
                ### Some error here
                print("idk")
                break
        else:
            levels.append(None)
    
df["Level"] = levels

In [ ]:
ask = Counter(df[(df['Labels']=='c') & (df['Dir'] == -1)]["Level"])
buy = Counter(df[(df['Labels']=='c') & (df['Dir'] == 1)]["Level"])
buy = buy[::-1]
for i in range(5):
    plt.bar(i, buy[i])
for i in range(5,11):
    plt.bar(i, ask[i])
plt.show()

In [ ]:
from collections import Counter

counts = Counter(df["Labels"])
print(counts)
print("{:0.1f}%".format(100*counts['c']/(counts['la'] + counts['lb'])))

In [ ]:
oids = list(set(df["OrderID"]))
from collections import Counter
#print(Counter(oids).most_common(100))

for oid in oids[1:]:
    history = df[df["OrderID"] == oid]
    if (len(history.index) > 2):
        break

print("Immediately Before:")
print("Bid:", df.iloc[107887]["P_bid_1"]/1e5, df.iloc[107887]["V_bid_1"])
print("Ask:", df.iloc[107887]["P_ask_1"]/1e5, df.iloc[107887]["V_ask_1"])
# print("Order:", history.iloc[17888]["P"]/1e5, history.iloc[17888]["V"], history.iloc[1]["Dir"])

print("\nCurrently:")
print("Bid:", history.iloc[1]["P_bid_1"]/1e5, history.iloc[1]["V_bid_1"])
print("Ask:", history.iloc[1]["P_ask_1"]/1e5, history.iloc[1]["V_ask_1"])
print("Order:", history.iloc[1]["P"]/1e5, history.iloc[1]["V"], history.iloc[1]["Dir"])
history[["OrderID","Type","P","V","Dir","P_bid_1","V_bid_1","P_ask_1","V_ask_1","Labels"]].head()

### Are all of the deleted orders fully deleted??
##### YES!

In [ ]:
find = 0
for oid in oids[1:]:
    history = df[df["OrderID"] == oid]
    labels = list(history["Labels"])
    if (('c' in labels) and (labels.count('c') > 2)):
#         find += 1
#         if (find == 2):
#             break
        break
        
history[["OrderID","Type","P","V","Dir","P_bid_1","V_bid_1","P_ask_1","V_ask_1","Labels"]].head(100)

In [ ]:
colnames = ["OrderID", "P", "Dir", "bV", "dV", "bLevel", "dLevel", "bLabel", "dLabel"]

uoids = []
for oid in oids[1:]:
    history = df[df["OrderID"] == oid].reset_index()
    birth = history.iloc[0]
    death = history.iloc[-1]
    uoids.append([oid, birth["P"], birth["Dir"], birth["V"], death["V"], ])
    break
history.head()

In [ ]:
# df_sens = ComputeTimeInsenstiveSet(df)
# df_sens.columns.values.tolist()

In [ ]:
# df_sensInsens = ComputeTimeSensitiveSet(df)

In [ ]:
# df_sensInsens.head(5)